In [38]:
from keplergl import KeplerGl
import json
import networkx
import uuid

censimento = json.load(open('dataset_completo.geojson'))
    
#Load Map config
%run 'Censimento_Milano_2011/config.py'

map = KeplerGl(data={"censimento_milano_2011": censimento},config=config)
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'k75ulfp', 'type': …

In [74]:
#Sezione-based randomic family generator

settori = []
id_familiare = 1

for i in censimento.get("features"):
    if (i.get("properties").get("POP_2010") == 0 or i.get("indicatori").get("1") == '' or i.get("indicatori").get("1") == "0"):
        continue
    print("Codice sezione:",i.get("properties").get("SEZ"))
    POP_2010 = i.get("properties").get("POP_2010")
    edifici = round(POP_2010/float(i.get("indicatori").get("1")))
    tot_appartamenti = round(edifici*float(i.get("indicatori").get("3")))
    print("Popolazione:",POP_2010)
    print("Numero di edifici ad uso residenziale:", edifici)
    print("Numero di abitazioni totali ad uso residenziale:", tot_appartamenti)
    if (tot_appartamenti!=0):
        avg_ppa = POP_2010/tot_appartamenti
        print("Numero di abitanti per appartamento ad uso residenziale:", avg_ppa)
        print("Numero medio di componenti delle famiglie:", i.get("indicatori").get("39"))
    #print("Percentuale di 65+ che vivono da soli:",i.get("indicatori").get("17"))
    #print("Percentuale di bambini in età prescolare:",i.get("indicatori").get("8"))

    #Percentuali esterne alla fascia 15-64
    if (i.get("indicatori").get("25") != '' and i.get("indicatori").get("26") != ''):
        young = float(i.get("indicatori").get("25"))
        old = float(i.get("indicatori").get("26"))
        Y = old + young

        X = (100+Y)/100
        mid_percent = 100/X
        old_percent = old/X
        young_percent = young/X

        anziani = round((POP_2010/100)*(old_percent))
        giovani = round((POP_2010/100)*(young_percent))
        attivi = round((POP_2010/100)*mid_percent)

        #Clone POP_2010 and households for counting purposes
        popolazione = POP_2010
        appartamenti = tot_appartamenti

        print("Abitanti nella fascia 15-64:", attivi)
        #print("Abitanti fuori dalla fascia 15-64:",anziani+giovani)
        print("Abitanti < 15:", giovani)
        print("Abitanti > 65:", anziani)
        print("Over 65+ che vivono da soli:",round(((round((POP_2010/100)*(old_percent)))/100*float(i.get("indicatori").get("27")))))
        print()

        settore = []

        #65+ old solo families generator
        for j in range (0,round(((round((POP_2010/100)*(old_percent)))/100*float(i.get("indicatori").get("27"))))):
            fam = {}
            members = []
            obj = {}
            obj["uuid"] = uuid.uuid4()
            obj["age"] = "old"
            fam["family_id"] = id_familiare
            members.append(obj)
            fam["members"] = members
            id_familiare = id_familiare + 1
            anziani = anziani - 1
            settore.append(fam)

        #Remove created families from 65+
        popolazione = popolazione - round(((round((POP_2010/100)*(old_percent)))/100*float(i.get("indicatori").get("27"))))
        appartamenti = appartamenti - round(((round((POP_2010/100)*(old_percent)))/100*float(i.get("indicatori").get("27"))))

        #Families with children one parent generator
        if (i.get("indicatori").get("14") != "0"):
            max_fam = (giovani/100)*float(i.get("indicatori").get("14"))
            num = round(max_fam/avg_ppa)
            #NUM = Number of families with one parent
            for k in range (0,num):

                #Members removal code
                kids_number = round(max_fam/num)
                max_fam = max_fam - kids_number
                POP_2010 = POP_2010 - (kids_number + 1)
                giovani = giovani - kids_number 
                fam = {}
                members = []
                obj = {}
                obj["uuid"] = uuid.uuid4()
                obj["age"] = "active"
                members.append(obj)

                for kid in range(0,kids_number):
                    obj = {}
                    obj["uuid"] = str(uuid.uuid4())
                    obj["age"] = "kid"
                    members.append(obj)

                fam["family_id"] = id_familiare
                fam["members"] = members
                id_familiare = id_familiare + 1

                attivi = attivi - num
                popolazione = popolazione - len(fam.get("members"))
                settore.append(fam)

        if (giovani != 0):
            #Max families members / mean familiy size
            max_componenti = round(giovani*3)
            avg_fam = round(max_componenti/avg_ppa)

            if (avg_fam > giovani):
                num = giovani
            else:
                num = avg_fam

            #NUM = Number of families with two parents
            mean_kpf = round(giovani/num)
            for k in range(0,num):
                
                fam = {}
                members = []

                #Add parents
                for parent in range(0,2):
                    obj = {}
                    obj["uuid"] = str(uuid.uuid4())
                    obj["age"] = "mid"
                    members.append(obj)

                if (giovani < mean_kpf):
                    kids = giovani
                else: 
                    kids = mean_kpf

                for kid in range(0,kids):
                    obj = {}
                    obj["uuid"] = str(uuid.uuid4())
                    obj["age"] = "kid"
                    members.append(obj)
                
                POP_2010 = POP_2010 - (2+kids)
                giovani = giovani - kids
                attivi = attivi - 2

                fam["family_id"] = id_familiare
                fam["members"] = members
                id_familiare = id_familiare + 1
                appartamenti = appartamenti - 1 
                popolazione = popolazione - len(fam.get("members"))
                settore.append(fam)


        #Generate remaining families
        #65+ families
        for index in range (0, round(anziani/2)):
            fam = {}
            members = []

            if (anziani == 1):
                #1 old, 1 mid
                obj = {}
                obj["uuid"] = str(uuid.uuid4())
                obj["age"] = "old"
                members.append(obj)
                obj = {}
                obj["uuid"] = str(uuid.uuid4())
                obj["age"] = "mid"
                members.append(obj)
                anziani = anziani - 1
                attivi = attivi - 1
            else:
                #2 old
                for parent in range(0,2):
                    obj = {}
                    obj["uuid"] = str(uuid.uuid4())
                    obj["age"] = "old"
                    members.append(obj)
                anziani = anziani - 2

            fam["family_id"] = id_familiare
            fam["members"] = members
            id_familiare = id_familiare + 1
            appartamenti = appartamenti - 1 
            popolazione = popolazione - len(fam.get("members"))
            settore.append(fam)

        #Active families
        for index in range (0, round(attivi/2)):
            fam = {}
            members = []

            if (attivi == 1):
                obj = {}
                obj["uuid"] = str(uuid.uuid4())
                obj["age"] = "mid"
                members.append(obj)
                attivi = attivi - 1
            else:
                #2 old
                for parent in range(0,2):
                    obj = {}
                    obj["uuid"] = str(uuid.uuid4())
                    obj["age"] = "mid"
                    members.append(obj)
                attivi = attivi - 2

            fam["family_id"] = id_familiare
            fam["members"] = members
            id_familiare = id_familiare + 1
            appartamenti = appartamenti - 1
            popolazione = popolazione - len(fam.get("members")) 
            settore.append(fam)


Codice sezione: 1
Popolazione: 15.0
Numero di edifici ad uso residenziale: 1
Numero di abitazioni totali ad uso residenziale: 8
Numero di abitanti per appartamento ad uso residenziale: 1.875
Numero medio di componenti delle famiglie: 1.75
Abitanti nella fascia 15-64: 6
Abitanti < 15: 1
Abitanti > 65: 8
Over 65+ che vivono da soli: 2

Codice sezione: 2
Popolazione: 21.0
Numero di edifici ad uso residenziale: 2
Numero di abitazioni totali ad uso residenziale: 9
Numero di abitanti per appartamento ad uso residenziale: 2.3333333333333335
Numero medio di componenti delle famiglie: 2.4
Abitanti nella fascia 15-64: 14
Abitanti < 15: 4
Abitanti > 65: 3
Over 65+ che vivono da soli: 0

Codice sezione: 3
Popolazione: 22.0
Numero di edifici ad uso residenziale: 2
Numero di abitazioni totali ad uso residenziale: 18
Numero di abitanti per appartamento ad uso residenziale: 1.2222222222222223
Numero medio di componenti delle famiglie: 1.11
Abitanti nella fascia 15-64: 7
Abitanti < 15: 0
Abitanti > 65:

ZeroDivisionError: division by zero